<a href="https://colab.research.google.com/github/Sil477/Politecnico/blob/main/Clase6_HugginFace_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1 - Pipeline

In [1]:
!pip install transformers -q

[Librería Transformers](https://github.com/huggingface/transformers)

In [2]:
from transformers import pipeline

In [3]:
# tarea de calsficiación
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [4]:
res = classifier("Me encantan las clases de Nechu, explica genial")
print(res)

[{'label': 'POSITIVE', 'score': 0.8972371220588684}]


In [5]:
res = classifier("El profesor es bueno todo será sencillo")
print(res)

[{'label': 'NEGATIVE', 'score': 0.6551787257194519}]


#### Selección del modelo

In [6]:
# seleccionamos el mismo modelo que tenemos por defecto
classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [7]:
res = classifier("El profesor es bueno todo será sencillo")
print(res)

[{'label': 'NEGATIVE', 'score': 0.6551787257194519}]


Existe una amplia variedad de 'pipelines': [lista de pipelines](https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.pipeline)

### 2 - Modelo y Tokenizer

El modelo lo hemos ejecutado con apenas una línea, pero realmente hay bastantes etapas que ocurren por debajo. En el siguiente código vamos a ver las más importantes.

In [8]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [9]:
# Primero veamos cuales son las etapas anteriores con el mismo modelo
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
#model_name = "pysentimiento/robertuito-sentiment-analysis"

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

classifier = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer
)

res = classifier("El profesor es bueno todo será sencillo")
print(res)

[{'label': 'NEGATIVE', 'score': 0.6551787257194519}]


### 3 - ¿Para qué sirve el tokenizer?

Codificación (antes del LLM)

In [11]:
secuencia = "El profesor es bueno todo será sencillo"
res = tokenizer(secuencia)
print(res)

{'input_ids': [101, 3449, 11268, 2229, 2953, 9686, 20934, 16515, 28681, 2080, 26358, 12411, 6895, 7174, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


Paso a paso

In [12]:
tokens = tokenizer.tokenize(secuencia)
print(tokens)

['el', 'prof', '##es', '##or', 'es', 'bu', '##eno', 'tod', '##o', 'sera', 'sen', '##ci', '##llo']


In [13]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

[3449, 11268, 2229, 2953, 9686, 20934, 16515, 28681, 2080, 26358, 12411, 6895, 7174]


Decodificar (después del LLM

In [14]:
tokenizer.decode(res['input_ids'])

'[CLS] el profesor es bueno todo sera sencillo [SEP]'

### 4 - Guardar modelo y tokenizer en local

In [15]:
model_path = ("./modelo")
tokenizer.save_pretrained(model_path)
model.save_pretrained(model_path)

In [16]:
tokenizer_local = AutoTokenizer.from_pretrained(model_path)
model_local = AutoModelForSequenceClassification.from_pretrained(model_path)

### 5 - Pytorch

También compatible con tensorflow

In [17]:
import torch
import torch.nn.functional as F

In [19]:
sentences = [
    "He estado deseando un curso así toda mi vida",
    "Me encanta HuggingFace"
]

Tokenizer

In [20]:
batch = tokenizer(
    sentences,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)
print(batch)

{'input_ids': tensor([[  101,  2002,  9765,  9365,  4078, 11219,  3527,  4895, 12731, 25301,
          2004,  2072, 28681,  2050,  2771, 19830,   102],
        [  101,  2033,  4372,  9336,  2696, 17662, 12172,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


Modelo

In [21]:
with torch.no_grad():
    outputs = model(**batch)
    predictions = F.softmax(outputs.logits, dim=1)
    labels = torch.argmax(predictions, dim=1)

In [22]:
# Logits
print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[ 1.3327, -1.1108],
        [-2.7697,  2.9255]]), hidden_states=None, attentions=None)


In [23]:
# Neg, Neu, Pos
print(predictions)

tensor([[0.9201, 0.0799],
        [0.0034, 0.9966]])


In [24]:
# etiquetas
print(labels)

tensor([0, 1])


In [25]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 